In [163]:
import os
import numpy as np
import cv2 as cv
import mediapipe as mp
import glob
import csv

import random
from PIL import Image,ImageFont,ImageDraw,features
import pyarabic.araby as araby
from keras.models import load_model
import PIL

In [164]:
print (features.check('raqm'))


True


In [165]:
root_dir = 'random_data\\'
aug_dir =root_dir + 'aug_images\\'
backgr_dir = root_dir + 'backgrounds\\'
cards = root_dir +'cards\\'
empty_card = root_dir + 'empty_card\\'
faces_dir = root_dir + 'humans\\'
arabic_names = root_dir + 'arabic_names\\'
dataset_root = 'dataset\\'
ids_ds = dataset_root + 'ids\\'
names_ds = dataset_root + 'names\\'
img_csv = dataset_root + 'image_csv\\'
dataset_gen_cards = dataset_root + 'gen_cards\\'
females_images = root_dir + 'faces\\' +'Female Faces'
males_images = root_dir + 'faces\\' +'Male Faces'


In [166]:
try:
    os.makedirs(dataset_gen_cards)
except:
    pass

In [167]:
try:
    os.makedirs(females_images)
    os.makedirs(males_images)
except:
    pass

In [168]:
def male_female_names(path):
    males = []
    females = []
    names = glob.glob(path+'\\*.csv')
    for csv_file_name in names:
        with open(csv_file_name,'r',encoding='utf-8') as f:
            csv_reader = csv.reader(f)
            next(csv_reader)
            for row in csv_reader:
                if row[1].lower() =='f':
                    females.append(row[0])
                elif row[1].lower() =='m':
                    males.append(row[0])
    return males,females

In [169]:
males, females = male_female_names(arabic_names)

In [170]:
def name_generator(gender):
    first_name = ''
    if gender.lower() == 'f':
        first_name = random.choice(females)
    else:
        first_name = random.choice(males)
    second_name = random.choice(males)
    third_name = random.choice(males)
    forth_name = random.choice(males)
    full_name = first_name +'\n' +second_name +' ' +third_name +' ' +forth_name
    
    return araby.normalize_alef(full_name).replace('ؤ','و')
    

In [171]:
def image_formater(cv_image):
    width = 332
    height = 444
    dim = (width,height)
    resized = cv.resize(cv_image,dim,interpolation = cv.INTER_LANCZOS4)
    blur = cv.blur(resized,ksize=(5,5))
    gray = cv.cvtColor(blur, cv.COLOR_BGR2GRAY)
    return gray

In [172]:
def generate_card_number():
    numbers_list = list(araby.NUMBERS_EAST)
    num = ''

    for _ in range(2):
        num += random.choice(numbers_list) 
    num += ' '
 
    for _ in range(3):
        num += random.choice(numbers_list)
    num += ' '
    for _ in range(2):
        num += random.choice(numbers_list)
        num += ' '
    for _ in range(2):
        num += random.choice(numbers_list)
    num += ' '
    for _ in range(2):
        num += random.choice(numbers_list)
    num += ' '

    for _ in range(2):
        num += random.choice(numbers_list)
    num += ' '
    num = num+ random.choice(numbers_list) + ' '

    return num

In [173]:
csv_ds = []
csv_ds.append(['image_location','name','id','person_box_x1','person_box_x2','name_box_x1','name_box_x2', 'id_box_x1','id_box_x2'])

In [174]:
def point_to_string(point):
    return str(point[0])+','+str(point[1])

In [175]:
def id_card_generator(path,gender):
    img_point = (29,56)
    card_path = empty_card + 'empty_id.jpg'
    c = Image.open(card_path)
    font_path = 'simpo.ttf'
    font = ImageFont.truetype('Simplified Arabic Bold.ttf',44)
    font_num = ImageFont.truetype('Simplified Arabic Bold.ttf',55)
    names = glob.glob(path+'\*.jpg')
    index = 0
    for image_location in names:
        try:   
            temp = c.copy()
            im = cv.imread(image_location)
            grey_image = image_formater(im)
            grey_image = Image.fromarray(grey_image)
            temp.paste(grey_image,img_point)
            draw = ImageDraw.Draw(temp)
            p_name = name_generator(gender)
            width ,height = draw.textsize(p_name,font=font,direction='rtl')
            number = generate_card_number()
            name_point = (1277 -width-70,250)
            num_point = (650,635)
        
            num_width ,num_height = draw.textsize(number,font=font_num,direction='rtl')
            name_rect = ((1277 -width-80,240),(1277-60,280+height))
            number_rect = ((640,630),(660+num_width,640+num_height))
            img_rect = ((25,50),(25+332,50+444))
            draw.text(name_point,p_name,fill=(0,0,0),font=font,align='right')
            draw.text(num_point,number,fill=(0,0,0),font=font_num,direction='rtl',align='right')
            location = dataset_gen_cards +gender + str(index)+'.jpg'
            csv_ds.append([location,p_name.replace('\n',' '),number,point_to_string(img_rect[0]),point_to_string(img_rect[1]),point_to_string(name_rect[0]),point_to_string(name_rect[1]),point_to_string(number_rect[0]),point_to_string(number_rect[1]),])

            temp.save(location)
            index +=1
        except:
            pass



In [176]:
id_card_generator(males_images,'m')
id_card_generator(females_images,'f')

C:\Users\molot\AppData\Local\Temp\ipykernel_21608\422519369.py:19: DeprecationWarning: textsize is deprecated and will be removed in Pillow 10 (2023-07-01). Use textbbox or textlength instead.
  width ,height = draw.textsize(p_name,font=font,direction='rtl')
C:\Users\molot\AppData\Local\Temp\ipykernel_21608\422519369.py:24: DeprecationWarning: textsize is deprecated and will be removed in Pillow 10 (2023-07-01). Use textbbox or textlength instead.
  num_width ,num_height = draw.textsize(number,font=font_num,direction='rtl')
C:\Users\molot\AppData\Local\Temp\ipykernel_21608\422519369.py:19: DeprecationWarning: textsize is deprecated and will be removed in Pillow 10 (2023-07-01). Use textbbox or textlength instead.
  width ,height = draw.textsize(p_name,font=font,direction='rtl')
C:\Users\molot\AppData\Local\Temp\ipykernel_21608\422519369.py:24: DeprecationWarning: textsize is deprecated and will be removed in Pillow 10 (2023-07-01). Use textbbox or textlength instead.
  num_width ,num_h

In [177]:
try:
    os.makedirs(img_csv)
except:
    pass
with open(img_csv + 'ds_images.csv', 'w',encoding='utf-8') as csvfile: 
    # creating a csv writer object 
    csvwriter = csv.writer(csvfile) 

        
    csvwriter.writerows(csv_ds)

In [178]:
try:
    os.makedirs(ids_ds)
    os.makedirs(names_ds)
except:
    pass

In [182]:
def feature_extractor(row):
    image_path = row[0]
    name = row[1]
    id = row[2]
    # left top right bottom
    im = Image.open(image_path)
    point_1 = tuple(row[7].split(','))
    point_2 = tuple(row[8].split(','))

    im1 = im.crop((int(point_1[0]), int(point_1[1]), (int(point_2[0])), int(point_2[1])))
    im1.save(ids_ds + id+'.jpg')
    point_3 = tuple(row[5].split(','))
    point_4 = tuple(row[6].split(','))
    im2 = im.crop((int(point_3[0]), int(point_3[1]), (int(point_4[0])), int(point_4[1])))
    im2.save(names_ds + name+'.jpg')

In [183]:
def ids_names_generator():
    ds = csv_ds[1:]
    for row in ds:
        feature_extractor(row)

In [184]:
ids_names_generator()